In [22]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
data = pd.read_csv('data/ALLFLOWMETER_HIKARI2022.csv', index_col=0)

In [9]:
data.head()

,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,...,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,attack_category,Label
0,Cmu9v81jToQyRF1gbk,184.0.48.168,38164,184.0.48.150,50443,0 days 00:00:00.000060,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,64240,0,64240,0,Benign,0
1,CO21hl3TWkuXTOgajk,184.0.48.169,43068,184.0.48.150,50443,0 days 00:00:00.000083,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,64240,0,64240,0,Benign,0
2,CBLJ6L19FP0MfYX7Oh,184.0.48.124,5678,255.255.255.255,5678,0 days 00:01:59.996602,3,0,3,0,...,5.999912e+07,1.199966e+08,5.999830e+07,1156.846698,0,0,0,0,Benign,0
3,ChTG451zJ7hUYOcqje,184.0.48.124,5678,255.255.255.255,5678,0 days 00:00:59.996909,2,0,2,0,...,5.999691e+07,5.999691e+07,5.999691e+07,0.000000,0,0,0,0,Benign,0
4,Cn9y6E2KVxzQbs5wjc,184.0.48.124,5678,255.255.255.255,5678,0 days 00:00:59.992130,2,0,2,0,...,5.999213e+07,5.999213e+07,5.999213e+07,0.000000,0,0,0,0,Benign,0


In [4]:
len(set(data['originh']))

5803

In [11]:
len(data)

228253

In [12]:
unique_originh = list(set(data['originh']))
unique_responh = list(set(data['responh']))
unique_all = list(set(unique_originh + unique_responh))

In [15]:
nodes_ip = np.random.choice(unique_all, 20)

In [21]:
data[(data['originh'].isin(nodes_ip)) | (data['responh'].isin(nodes_ip))]

,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,...,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,attack_category,Label
2,CBLJ6L19FP0MfYX7Oh,184.0.48.124,5678,255.255.255.255,5678,0 days 00:01:59.996602,3,0,3,0,...,5.999912e+07,1.199966e+08,5.999830e+07,1156.846698,0,0,0,0,Benign,0
3,ChTG451zJ7hUYOcqje,184.0.48.124,5678,255.255.255.255,5678,0 days 00:00:59.996909,2,0,2,0,...,5.999691e+07,5.999691e+07,5.999691e+07,0.000000,0,0,0,0,Benign,0
4,Cn9y6E2KVxzQbs5wjc,184.0.48.124,5678,255.255.255.255,5678,0 days 00:00:59.992130,2,0,2,0,...,5.999213e+07,5.999213e+07,5.999213e+07,0.000000,0,0,0,0,Benign,0
5,CDS4Hrw61MNCJg4Xc,184.0.48.124,5678,255.255.255.255,5678,0 days 00:00:00,1,0,1,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0,0,0,0,Benign,0
6,CE3evwqmFHImkXnY2,184.0.48.124,5678,255.255.255.255,5678,0 days 00:00:00,1,0,1,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0,0,0,0,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60968,Ckg6P54AItbJ94on1d,114.79.32.204,54628,184.0.48.169,443,0 days 00:00:00.200060,12,12,3,6,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,13720,65160,17777,64,Benign,0
71486,Ctuano25vXNpQeMAOg,114.79.32.204,54630,184.0.48.169,443,0 days 00:00:00.173915,11,11,3,6,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,13720,65160,17777,64,Benign,0
85048,CgAxJX26tMwxpQoKh3,114.79.32.204,54634,184.0.48.169,443,0 days 00:00:00.185774,13,11,3,6,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,13720,65160,17777,64,Benign,0
97077,Cj48wp4D3jwJ1k9x5,114.79.32.204,54722,184.0.48.169,443,0 days 00:00:16.120921,15,11,5,6,...,1.529914e+07,1.529914e+07,1.529914e+07,0.000000,12920,65160,18368,64,Benign,0


In [19]:
data[data['responh'].isin(data['originh'])]

,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,...,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,attack_category,Label
0,Cmu9v81jToQyRF1gbk,184.0.48.168,38164,184.0.48.150,50443,0 days 00:00:00.000060,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64240,0,64240,0,Benign,0
1,CO21hl3TWkuXTOgajk,184.0.48.169,43068,184.0.48.150,50443,0 days 00:00:00.000083,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64240,0,64240,0,Benign,0
172,CP0SgD3mSs2SQFEtPf,184.0.48.53,59370,184.0.48.150,50443,0 days 00:00:00.000061,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,29200,0,29200,0,Benign,0
1142,Cyk0iWZI3IbyerRDf,184.0.48.53,59372,184.0.48.150,50443,0 days 00:00:00.000059,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,29200,0,29200,0,Benign,0
1292,CcQ2HF4p0WM3KXfNId,184.0.48.171,49426,184.0.48.150,50443,0 days 00:00:00.000047,1,1,0,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64240,0,64240,0,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29201,ClVPuY3gp0KZP0Euu1,203.178.143.112,32280,184.0.48.169,443,0 days 00:00:03.205931,14,14,5,7,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64240,65160,501,64,Bruteforce-XML,1
42824,Cps0484off68GAE3E3,203.178.143.112,32744,184.0.48.169,443,0 days 00:00:05.803172,15,14,5,7,...,5.306690e+06,5.306690e+06,5.306690e+06,0.000000e+00,64240,65160,501,64,Bruteforce-XML,1
51787,CYwtZ2UXjNpNtMlI,203.178.143.112,32282,184.0.48.169,443,0 days 00:00:20.699521,15,14,5,7,...,1.936086e+07,1.936086e+07,1.936086e+07,0.000000e+00,64240,65160,501,64,Bruteforce-XML,1
64053,CZIQDW3XrAPNmlJNC9,203.178.143.112,32746,184.0.48.169,443,0 days 00:00:03.361322,14,14,5,7,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64240,65160,501,64,Bruteforce-XML,1


In [23]:
G = nx.Graph()

In [ ]:
for i, ip in enumerate(unique_all):
    G.nodes[i]['ip'] = ip

In [24]:
for i, row in data.iterrows():
    G.add_edge(row['originh'], row['responh'])

In [40]:
list(G.adj['103.144.170.44'])

['184.0.48.169']

In [31]:
sample_node = np.random.choice(unique_originh, 1)[0]

In [45]:
sample_node

'103.144.170.44'

In [49]:
G.adj['184.0.48.169']

AtlasView({'184.0.48.150': {}, '140.213.164.207': {}, '103.247.15.246': {}, '119.2.52.53': {}, '182.2.40.15': {}, '184.0.48.171': {}, '140.213.177.193': {}, '180.254.3.47': {}, '103.107.85.11': {}, '182.2.36.15': {}, '36.79.76.33': {}, '36.71.86.37': {}, '149.110.58.11': {}, '180.242.110.152': {}, '114.10.22.85': {}, '180.243.62.206': {}, '110.136.180.161': {}, '182.2.75.248': {}, '36.73.133.102': {}, '36.71.82.222': {}, '20.225.41.143': {}, '182.2.70.17': {}, '36.80.178.250': {}, '114.10.4.5': {}, '36.71.90.51': {}, '36.71.83.41': {}, '103.144.170.28': {}, '140.213.160.159': {}, '10.255.254.228': {}, '103.144.170.15': {}, '36.81.44.40': {}, '114.10.11.199': {}, '125.163.251.198': {}, '111.68.29.134': {}, '103.143.196.7': {}, '140.213.160.254': {}, '182.2.41.166': {}, '182.2.74.251': {}, '182.2.37.252': {}, '140.213.160.219': {}, '125.160.102.101': {}, '114.142.168.12': {}, '114.79.32.109': {}, '192.168.22.9': {}, '114.10.16.114': {}, '36.80.175.253': {}, '36.80.210.242': {}, '103.65.2

In [51]:
hop = 5
nodes = [sample_node]
frontier = [sample_node]
visited = [sample_node]
for h in range(hop):
    new_frontier = []
    for f in frontier:
        neighbours = [x for x in list(G.adj[f]) if x not in visited][:5]
        print(neighbours)
        if len(neighbours) == 0:
            continue
        nodes += neighbours
        new_frontier += neighbours
        visited.append(f)
    frontier = new_frontier
        

['184.0.48.169']
['184.0.48.150', '140.213.164.207', '103.247.15.246', '119.2.52.53', '182.2.40.15']
['184.0.48.168', '184.0.48.53', '184.0.48.171', '184.0.48.27', '184.0.48.23']
[]
[]
[]
[]
['172.17.0.2']
[]
['172.18.0.6']
[]
[]
['185.199.108.133', '185.199.110.133', '185.199.109.133']
[]


In [53]:
nodes

['103.144.170.44',
 '184.0.48.169',
 '184.0.48.150',
 '140.213.164.207',
 '103.247.15.246',
 '119.2.52.53',
 '182.2.40.15',
 '184.0.48.168',
 '184.0.48.53',
 '184.0.48.171',
 '184.0.48.27',
 '184.0.48.23',
 '172.17.0.2',
 '172.18.0.6',
 '185.199.108.133',
 '185.199.110.133',
 '185.199.109.133']

In [57]:
G.subgraph(nodes).edges

EdgeView([('184.0.48.169', '184.0.48.150'), ('184.0.48.169', '140.213.164.207'), ('184.0.48.169', '103.247.15.246'), ('184.0.48.169', '119.2.52.53'), ('184.0.48.169', '182.2.40.15'), ('184.0.48.169', '184.0.48.171'), ('184.0.48.169', '184.0.48.168'), ('184.0.48.169', '103.144.170.44'), ('172.17.0.2', '185.199.108.133'), ('172.17.0.2', '185.199.110.133'), ('172.17.0.2', '185.199.109.133'), ('172.17.0.2', '184.0.48.168'), ('184.0.48.168', '184.0.48.150'), ('185.199.108.133', '184.0.48.150'), ('184.0.48.53', '184.0.48.150'), ('185.199.109.133', '184.0.48.150'), ('184.0.48.150', '184.0.48.171'), ('184.0.48.150', '184.0.48.27'), ('184.0.48.150', '184.0.48.23'), ('184.0.48.150', '185.199.110.133'), ('172.18.0.6', '184.0.48.171')])